# 3. Creating a Simple Agent

In this notebook, we will learn how to create a simple agent using the `google-adk` library. Agents are AI systems that can perform tasks by reasoning, planning, and interacting with tools or APIs.

## Step 1: Import Required Libraries

We will start by importing the necessary libraries and defining constants for our agent.

In [20]:
import json
from google.adk.agents import LlmAgent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types
from pydantic import BaseModel, Field

# Define constants
APP_NAME = "agent_comparison_app"
USER_ID = "test_user_456"
SESSION_ID_TOOL_AGENT = "session_tool_agent_xyz"
SESSION_ID_SCHEMA_AGENT = "session_schema_agent_xyz"
MODEL_NAME = "gemini-2.0-flash"

## Step 2: Define Input and Output Schemas

We will define the input schema for both agents and the output schema for the second agent. These schemas ensure that the agents understand the structure of the data they process.

In [21]:
class CountryInput(BaseModel):
    country: str = Field(description="The country to get information about.")

class CapitalInfoOutput(BaseModel):
    capital: str = Field(description="The capital city of the country.")
    population_estimate: str = Field(description="An estimated population of the capital city.")

## Step 3: Define a Tool

The first agent will use a tool to retrieve the capital city of a country. Here, we define the tool as a Python function.

In [22]:
def get_capital_city(country: str) -> str:
    """Retrieves the capital city of a given country."""
    country_capitals = {
        "united states": "Washington, D.C.",
        "canada": "Ottawa",
        "france": "Paris",
        "japan": "Tokyo"
    }
    return country_capitals.get(country.lower(), f"Sorry, I couldn't find the capital for {country}.")

## Step 4: Configure Agents

We will now configure two agents:
1. An agent that uses the `get_capital_city` tool.
2. An agent that provides structured information without using tools.

In [26]:
capital_agent_with_tool = LlmAgent(
    model=MODEL_NAME,
    name="capital_agent_tool",
    description="Retrieves the capital city using a specific tool.",
    instruction="""You are a helpful agent that provides the capital city of a country using a tool.
The user will provide the country name in a JSON format like {\"country\": \"country_name\"}.
1. Extract the country name.
2. Use the `get_capital_city` tool to find the capital.
3. Respond clearly to the user, stating the capital city found by the tool.""",
    tools=[get_capital_city],
    input_schema=CountryInput,
    output_key="capital_tool_result",
)


In [ ]:

instruction = f"""You are an agent that provides country information.
The user will provide the country name in a JSON format like {{\"country\": \"country_name\"}}.
Respond ONLY with a JSON object matching this exact schema:
{json.dumps(CapitalInfoOutput.model_json_schema(), indent=2)}
Use your knowledge to determine the capital and estimate the population. Do not use any tools.
"""

structured_info_agent_schema = LlmAgent(
    model=MODEL_NAME,
    name="structured_info_agent_schema",
    description="Provides capital and estimated population in a specific JSON format.",
    instruction=instruction,
    input_schema=CountryInput,
    output_schema=CapitalInfoOutput,
    output_key="structured_info_result",
    disallow_transfer_to_parent=True,
    disallow_transfer_to_peers=True,
)

## Step 5: Set Up Session Management and Runners

We will create sessions for each agent and set up runners to manage their interactions.

In [29]:
session_service = InMemorySessionService()

session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID_TOOL_AGENT)
session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID_SCHEMA_AGENT)

capital_runner = Runner(
    agent=capital_agent_with_tool,
    app_name=APP_NAME,
    session_service=session_service
)
structured_runner = Runner(
    agent=structured_info_agent_schema,
    app_name=APP_NAME,
    session_service=session_service
)

## Step 6: Define Agent Interaction Logic

We will define a function to interact with the agents and print their responses.

In [30]:
async def call_agent_and_print(runner_instance: Runner, agent_instance: LlmAgent, session_id: str, query_json: str):
    user_content = types.Content(role='user', parts=[types.Part(text=query_json)])
    async for event in runner_instance.run_async(user_id=USER_ID, session_id=session_id, new_message=user_content):
        if event.is_final_response() and event.content and event.content.parts:
            print(event.content.parts[0].text)

In [36]:
async def call_agent_and_print(
    runner_instance: Runner,
    agent_instance: LlmAgent,
    session_id: str,
    query_json: str
):
    """Sends a query to the specified agent/runner and prints results."""
    print(f"\n>>> Calling Agent: '{agent_instance.name}' | Query: {query_json}")

    user_content = types.Content(role='user', parts=[types.Part(text=query_json)])

    final_response_content = "No final response received."
    async for event in runner_instance.run_async(user_id=USER_ID, session_id=session_id, new_message=user_content):
        # print(f"Event: {event.type}, Author: {event.author}") # Uncomment for detailed logging
        if event.is_final_response() and event.content and event.content.parts:
            # For output_schema, the content is the JSON string itself
            final_response_content = event.content.parts[0].text

    print(f"<<< Agent '{agent_instance.name}' Response: {final_response_content}")

    current_session = session_service.get_session(app_name=APP_NAME,
                                                  user_id=USER_ID,
                                                  session_id=session_id)
    stored_output = current_session.state.get(agent_instance.output_key)

    # Pretty print if the stored output looks like JSON (likely from output_schema)
    print(f"--- Session State ['{agent_instance.output_key}']: ", end="")
    try:
        # Attempt to parse and pretty print if it's JSON
        parsed_output = json.loads(stored_output)
        print(json.dumps(parsed_output, indent=2))
    except (json.JSONDecodeError, TypeError):
         # Otherwise, print as string
        print(stored_output)
    print("-" * 30)

## Step 7: Run Interactions

Finally, we will test the agents by sending queries and observing their responses.

In [37]:
async def main():
    print("--- Testing Agent with Tool ---")
    await call_agent_and_print(capital_runner, capital_agent_with_tool, SESSION_ID_TOOL_AGENT, '{"country": "France"}')
    await call_agent_and_print(capital_runner, capital_agent_with_tool, SESSION_ID_TOOL_AGENT, '{"country": "Canada"}')

    print("\n\n--- Testing Agent with Output Schema (No Tool Use) ---")
    await call_agent_and_print(structured_runner, structured_info_agent_schema, SESSION_ID_SCHEMA_AGENT, '{"country": "France"}')
    await call_agent_and_print(structured_runner, structured_info_agent_schema, SESSION_ID_SCHEMA_AGENT, '{"country": "Japan"}')

import asyncio
# run in synchronous context
await main()


--- Testing Agent with Tool ---

>>> Calling Agent: 'capital_agent_tool' | Query: {"country": "France"}


ValueError: Missing key inputs argument! To use the Google AI API, provide (`api_key`) arguments. To use the Google Cloud API, provide (`vertexai`, `project` & `location`) arguments.

## Recap & Next Steps

You've just built and interacted with a basic agent!

**Key Concepts:**
*   **Tools:** Functions the agent can use.
*   **FunctionDeclaration:** Describing tools for the LLM.
*   **FunctionCall:** The LLM requesting a tool execution.
*   **FunctionResponse:** Providing the tool's result back to the LLM.
*   **Multi-turn Conversation:** The interaction often involves back-and-forth between the user, LLM, and tools.

> #### 🎁 Bonus exercises 📝
> - **Add More Tools:** Create another tool (e.g., one that returns the current date) and add its `FunctionDeclaration` to the `tools` list.
> - **Error Handling:** What happens if the user asks for division by zero? How could you make the agent handle tool errors more gracefully?
> - **Complex Queries:** Try asking multi-step questions like "What is 5 plus 3, and then multiply the result by 2?". Does the agent handle it in one go or multiple steps?
> - **Explore Frameworks:** Libraries like LangChain or LlamaIndex provide higher-level abstractions for building more complex agents.